In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import plotly as py
import plotly.tools as tls
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [3]:
df4=pd.read_csv('/content/drive/MyDrive/Data_used_for_practice/daily-minimum-temperatures-in-me.csv', parse_dates=True)
df4.head()


,Date,Daily minimum temperatures
0,1/1/1981,20.7
1,1/2/1981,17.9
2,1/3/1981,18.8
3,1/4/1981,14.6
4,1/5/1981,15.8


In [4]:
df4.shape

(3650, 2)

In [5]:
df4.index = np.arange(1, len(df4) + 1) #to have the index row start at 1

In [6]:
### droppping rows beyond 2048
if len(df4) > 2048:
    df4 = df4.drop(range(2049,len(df4)+1))

df4

,Date,Daily minimum temperatures
1,1/1/1981,20.7
2,1/2/1981,17.9
3,1/3/1981,18.8
4,1/4/1981,14.6
5,1/5/1981,15.8
...,...,...
2044,8/7/1986,9.1
2045,8/8/1986,3.5
2046,8/9/1986,3.6
2047,8/10/1986,2.5


In [7]:
type(df4.index[0]) #so that index values that we have are ints

numpy.int64

In [8]:
df4 = df4.drop(['Date'], axis = 1)

In [11]:
df4 = df4.replace('\?','',regex=True).astype(float)

In [12]:
df4 = df4.astype('float')

In [13]:
### estimating length of one percent
one_cent = int(0.01*len(df4))
print("one_cent is", one_cent)

one_cent is 20


In [14]:
trim_end = 1
trim_start = 1

df4_orig = df4.copy()

if trim_end == 1:
    df4 = df4.drop(range(df4.index[-one_cent],df4.index[-1]+1))

if trim_start == 1:
    df4 = df4.drop(range(df4.index[0],one_cent+1))

df4

,Daily minimum temperatures
21,18.2
22,12.1
23,14.4
24,16.0
25,16.5
...,...
2024,7.6
2025,9.2
2026,9.7
2027,8.8


In [15]:

first_10_cent = df4[:10*one_cent]
last_10_cent = df4[-10*one_cent:]

# print(first_10_cent)
# print(last_10_cent)

### mean and sigma evaluation
mean_first10 = first_10_cent['Daily minimum temperatures'].mean()
mean_last10 = last_10_cent['Daily minimum temperatures'].mean()

sigma_first10 = first_10_cent['Daily minimum temperatures'].std()
sigma_last10 = last_10_cent['Daily minimum temperatures'].std()

#print(mean_first10, sigma_first10, mean_last10, sigma_last10)

LL_first10= mean_first10  - 6.0*sigma_first10
UL_first10= mean_first10  + 6.0*sigma_first10

LL_last10= mean_last10  - 6.0*sigma_last10
UL_last10= mean_last10  + 6.0*sigma_last10

print(mean_first10)
print(mean_last10)

print("12 sigma limits for first 10 percent = ", LL_first10, UL_first10) 
print("12 sigma limits for last 10 percent =", LL_last10, UL_last10)

11.348999999999995
11.463999999999999
12 sigma limits for first 10 percent =  -16.230082525798856 38.92808252579885
12 sigma limits for last 10 percent = -10.618880519993343 33.54688051999334


In [16]:
df4_orig

first1_outside12sigma = 0
last1_outside12sigma = 0

for i in range(1,one_cent+1,1):
    if df4_orig['Daily minimum temperatures'][i] < LL_first10 or df4_orig['Daily minimum temperatures'][i] > UL_first10:
        first1_outside12sigma += 1

for i in range(len(df4_orig)-one_cent+1,len(df4_orig)+1,1):
    if df4_orig['Daily minimum temperatures'][i] < LL_last10 or df4_orig['Daily minimum temperatures'][i] > UL_last10:
        last1_outside12sigma += 1

print(first1_outside12sigma, last1_outside12sigma)

0 0


In [17]:
df4_final = df4_orig.copy()

if last1_outside12sigma > 0:
    df4_final = df4_final.drop(range(df4_final.index[-one_cent],df4_final.index[-1]+1))
    

if first1_outside12sigma > 0:
    df4_final = df4_final.drop(range(df4_final.index[0],one_cent+1))

df4_final

,Daily minimum temperatures
1,20.7
2,17.9
3,18.8
4,14.6
5,15.8
...,...
2044,9.1
2045,3.5
2046,3.6
2047,2.5
